In [ ]:
# conn, import

In [ ]:
#fn_parse_into_trails_and_get_lickrate function for ROILick2DPSTHSpikes 
import datajoint as dj
import numpy as np
######
##call for the needed tables- 	
#	IMG.FrameStartTrail (keys are s_e_trail_bitcode, s_e_trail_frame=relative to the beginning of thre s.e., s_e_trail_end_frame), 
#	TRACKING.VideoGroomingTrail (if there was grooming and so we need to exclude the trail), 
#	EXP2.BehaviorTrail (parenting task protocol, trail instruction-l/r/no/go, earlylick-no/early, outcome-hit/miss/ignore), 
#	EXP2.BehaviorTrailEvent, 
#	EXP2.ActionEvent (key is action_event_time which tells the sec from trail start)


img= dj.schema('arseny_learning_imaging', locals())
imgframstarttrail1= dj.VirtualModule('IMG', 'arseny_learning_imaging')
imgframstarttrail= imgframstarttrail1.IMG.FramStartTrail

tracking= dj.schema('arseny_learning_tracking', locals())
tracking= dj.VirtualModule('TRACKING', 'arseny_learning_tracking')
grooming= tracking.VideoGroomingTrail

exp2= dj.schema('arseny_s1alm_experiment2', locals())
exp2vm= dj.VirtualModule ('EXP2', 'arseny_s1alm_experiment2')
behaviortrail= exp2vm.BehaviorTrial
behaviortrailevent= exp2vm.BehaviorTrailEvent
actionevent= exp2vm.ActionEvent

# change manualy time frame bin and flag_electric according to the analysis(creat a function to replace to manual method)
######

#
def fn_parse_into_trails_and_get_lickrate (key, frame_rate, time_bin, flag_electric_video):

key = {'key': ''}  # specify the key

TrialsStartFrame = (imgframstarttrail & key - grooming).fetch ('session_epoch_trial_start_frame', 'ORDER BY trial') 
trialnum=(imgframstarttrail & key - grooming).fetch ('trial','ORDER BY trial')

if len(TrialsStartFrame) == "0":
	 TrialsStartFrame= (imgframstarttrail & key- grooming).fetch ('session_epoch_file_start_frame', 'ORDER BY session_epoch_file_num')
	 trailnum=(behaviortrail & key - grooming).fetch ('trial','ORDER BY trial')
	 TrialsStartFrame= TrialsStartFrame(trailnum)


##flag_electric_video## determine in ROILick2DmapSpikes, choose 1 for electric and 2 for video
from ROILick2DmapSpikes import Flage_Electric_Video # change to the name of the origin for the varible
flagelectricvideo= Flage_Electric_Video

if flagelectricvideo == 1:
        lickvideo = [] #We align based on electric lickport, even if video does exist
    elif flagelectricvideo == 2: #We  try to align based on video, if it exists.
        # We align to the first video-detected lick after lickport movement
        lickvideo = (videonthlicktrail & key - grooming).fetch('lick_time_onset_relative_to_trial_start')# we use it to get the timing of licks after lickport entrance

## multiple options for alignment- test all and compare between them (appearance of lickport (first contact lick, go cue in this case), first lick-w or w/o lickport appearance)

# align to the first detected lick after go cue


gotime = (behaviortrialevent & key - grooming & 'trial_event_type="go"').fetch('trial_event_time')
lickelectric=(actionevent & key - grooming).fetch('*') # fetch all

start_file = np.zeros(len(trialnum))
end_file = np.zeros(len(trialnum))
lick_tr_times_relative_to_first_lick_after_go = []
lick_tr_total = np.zeros(len(trialnum))

for i_tr, trial in enumerate(trialnum):
    if len(lickvideo) > 0:
        all_licks = lickvideo[[lick['trial'] == trial for lick in lickvideo]]
        all_licks = [lick[0] for lick in all_licks]
        licks_after_go = [lick for lick in all_licks if lick > gotime[i_tr]]
    else: #use electric
        all_licks = lickelectric[[lick['trial'] == trial for lick in lickelectric]]
        all_licks = [lick[2] for lick in all_licks]
        licks_after_go = [lick for lick in all_licks if lick > gotime[i_tr]]


 
